<a href="https://colab.research.google.com/github/leolellisr/npl_natural_language_processing_projects/blob/main/05_Embeddings/05_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook de referência 

Nome: Leonardo de Lellis Rossi

https://app.neptune.ai/leolellisr/nlp-imbd-large/e/NIMBL-39/charts

# Fixando a seed

In [ ]:
import random
import torch
import numpy as np

In [ ]:
def set_seeds():
  random.seed(123)
  np.random.seed(123)
  torch.manual_seed(123)
  torch.cuda.manual_seed(123)
set_seeds()  

## Preparando Dados 

Primeiro, fazemos download do dataset:

In [ ]:
!wget -nc http://files.fast.ai/data/aclImdb.tgz 
!tar -xzf aclImdb.tgz

File ‘aclImdb.tgz’ already there; not retrieving.



## Carregando o dataset

Criaremos uma divisão de treino (80%) e validação (20%) artificialmente.

Nota: Evitar de olhar ao máximo o dataset de teste para não ficar enviseado no que será testado. Em aplicações reais, o dataset de teste só estará disponível no futuro, ou seja, é quando o usuário começa a testar o seu produto.

In [ ]:
import os
import random


def load_texts(folder):
    texts = []
    for path in os.listdir(folder):
        with open(os.path.join(folder, path)) as f:
            texts.append(f.read())
    return texts

x_train_pos = load_texts('aclImdb/train/pos')
x_train_neg = load_texts('aclImdb/train/neg')
x_test_pos = load_texts('aclImdb/test/pos')
x_test_neg = load_texts('aclImdb/test/neg')

x_train = x_train_pos + x_train_neg
x_test = x_test_pos + x_test_neg
y_train = [True] * len(x_train_pos) + [False] * len(x_train_neg)
y_test = [True] * len(x_test_pos) + [False] * len(x_test_neg)

# Embaralhamos o treino para depois fazermos a divisão treino/valid.
c = list(zip(x_train, y_train))
random.shuffle(c)
x_train, y_train = zip(*c)

n_train = int(0.8 * len(x_train))

x_valid = x_train[n_train:]
y_valid = y_train[n_train:]
x_train = x_train[:n_train]
y_train = y_train[:n_train]

print(len(x_train), 'amostras de treino.')
print(len(x_valid), 'amostras de desenvolvimento.')
print(len(x_test), 'amostras de teste.')

print('3 primeiras amostras treino:')
for x, y in zip(x_train[:3], y_train[:3]):
    print(y, x[:100])

print('3 últimas amostras treino:')
for x, y in zip(x_train[-3:], y_train[-3:]):
    print(y, x[:100])

print('3 primeiras amostras validação:')
for x, y in zip(x_valid[:3], y_test[:3]):
    print(y, x[:100])

print('3 últimas amostras validação:')
for x, y in zip(x_valid[-3:], y_valid[-3:]):
    print(y, x[:100])

20000 amostras de treino.
5000 amostras de desenvolvimento.
25000 amostras de teste.
3 primeiras amostras treino:
False Realistic movie,sure,except for the fact that the characters don't look like to be scared. When Bill
False DVD has become the equivalent of the old late night double-bill circuit, the last chance to catch ol
True this is the first of a two part back-story to the conflict between the machines and mankind in the M
3 últimas amostras treino:
False This is, without a doubt, the most offensive "chick flick" I have seen in years, if not ever. The wr
True I am going to go out on a limb, and actually defend "Shades of Grey" as a good clip-show episode, wh
True Many people know how it feels when a loved one is lost. The feelings of pain, grief and sorrow can b
3 primeiras amostras validação:
True Netflix should mention this short feature on the info for Silk Stockings. Superior in every way to t
True I've watched the first 17 episodes and this series is simply amazing! I haven

In [ ]:
sum([len(item.split()) for item in x_train])

4689303

# Imports

In [ ]:
%matplotlib inline
import numpy as np
from bs4 import BeautifulSoup
import torch
from torch.utils.data import DataLoader

import re
from collections import Counter, OrderedDict
import numpy as np

from torchtext.vocab import vocab
import matplotlib.pyplot as plt



In [ ]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
   print(torch. cuda. get_device_name(dev))
else: 
   dev = "cpu" 
print(dev)
device = torch.device(dev)

Tesla K80
cuda:0


# Shuffle DS

In [ ]:
c = list(zip(x_train, y_train))
d = list(zip(x_valid, y_valid))

random.shuffle(c)
x_valid, y_valid = zip(*d)
x_train, y_train = zip(*c)
len(x_train)

20000

# Tokenizer

In [ ]:
def tokenizer(sentence):
    #Beautiful soup to remove tags of sentences 
    removeTags = BeautifulSoup(sentence, 'html.parser') 

    # Getting list of words and remove numbers and ponctuation with regex
    removeNotWords = re.sub('\W+',' ', removeTags.text)
    regex = re.compile('\w+|[^\w\s]+')
    re_split = regex.findall(removeNotWords.lower())
    
    re_split_without_numbers = [word for word in re_split if not word.isnumeric()]
    return re_split_without_numbers
x_train_token = tokenizer(' '.join(x_train))

# Vocabulary

In [ ]:
# Size Vocabulary
size_vocab = 10000

# Count tokens
counterTokens = dict(Counter(x_train_token).most_common(size_vocab))

# Define vocabulary with torch.vocab
vocab_train = vocab(counterTokens, min_freq=1)
vocab_train.set_default_index(len(vocab_train))

print('len of train vocabulary is',len(vocab_train))

bow_pipeline = lambda x: [vocab_train[token] for token in tokenizer(x)]

len of train vocabulary is 10000


In [ ]:
print(bow_pipeline('This movie$#@ &*() is, amazing! fr33'))
bow_pipeline('Thats something Thats something !$ This movie$#@ &*() is, amazing!')

[9, 16, 5, 473, 10000]


[1585, 138, 1585, 138, 9, 16, 5, 473]

# Padding parameters 

In [ ]:
max_pad = 200
pad_idx = len(vocab_train)+1

# Dataset

In [ ]:
class Ex5_ds(torch.utils.data.Dataset):

    def __init__(self, x, y, mode):
        self.x = x
        self.y = y
        self.mode = mode

    def __len__(self):
        return len(self.y)

    def __getitem__(self, index):
        
        y_label = int(self.y[index]) # bool to int
        token_idx = bow_pipeline(self.x[index]) # Getting bow
        token_idx = token_idx[:max_pad] # Remove tokens when len(tokens)>max_pad
        
        if self.mode == 'bow_hist': # same as "Aula 4"
            all_tokens = torch.zeros(len(vocab_train)+1).long()
            count_dict = Counter(token_idx)
            dict_keys = list(count_dict.keys())
            dict_values = torch.tensor(list(count_dict.values())).long()
            all_tokens[dict_keys] = dict_values
            all_tokens = all_tokens.float()

        elif self.mode =='emb':
            if len(token_idx) < max_pad: # when embedding, if len(tokens)<max_pad, they're completed with pad_idx
                token_idx = token_idx + (max_pad - len(token_idx)) * [pad_idx]
            assert len(token_idx) == max_pad # verify if I get exactly max_pad tokens

            all_tokens = torch.tensor(token_idx).long()

        return all_tokens, y_label

# Install and config Neptune

In [ ]:
! pip install neptune-client

In [ ]:
import neptune.new as neptune

run = neptune.init(project='leolellisr/nlp-imbd-large', api_token='eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1NjY1YmJkZi1hYmM5LTQ3M2QtOGU1ZC1iZTFlNWY4NjE1NDQifQ==')

https://app.neptune.ai/leolellisr/nlp-imbd-large/e/NIMBL-39
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


Creating weights (W) and bias 2nd layer

For linear/embedding + relu + linear implementation, didn't get same results.

Needed to clone weights of bow_hist model and then .cat with zeros for padding.

In [ ]:
# Input size: vocab_train+2 (unknown+pad)
# Hidden size: 128
#set_seeds()

# Weights of 1st layer 
#fst_W = torch.rand(128, len(vocab_train)+2)

# Weights of 2nd layer 
#snd_W = torch.rand(2, 128)

# Bias 2nd layer 
#snd_bias = torch.rand(2)


# Models

In [ ]:
class Ex5_model(torch.nn.Module):
    def __init__(self, mode, input, hidden):
        super(Ex5_model, self).__init__()
        self.mode = mode
        if mode == 'bow_hist': # same as "Aula 4"
          self.fst_layer = torch.nn.Linear(input, hidden, bias=False, device=device)
          
          # didn't work
          #self.fst_layer.load_state_dict(OrderedDict([('weight',  fst_W[:,:-1])]))
        elif mode == 'emb':
          self.fst_layer = torch.nn.Embedding(input, hidden, device=device, padding_idx=pad_idx)        
          
          # didn't work
          #self.fst_layer.load_state_dict(OrderedDict([('weight',  fst_W.T)])) #same effect as _weight=fst_W.T in torch.nn.Embedding
        else: print("Invalid mode")

        self.snd_linear_layer = torch.nn.Linear(hidden, 2, device=device, bias=False)
        
        # didn't work
        #self.snd_linear_layer.load_state_dict(OrderedDict([('weight',  snd_W), ('bias', snd_bias)]))
                    
        self.relu = torch.nn.ReLU()
    
    def forward(self, x):
        x = self.fst_layer(x)
        if self.mode == 'emb': x = torch.sum(x, dim=1) 
        x = self.relu(x)
        x = self.snd_linear_layer(x)
        #x = self.relu(x)
        return x


In [ ]:
#class Ex5_model_emb(torch.nn.Module):
#    def __init__(self,input, hidden, fst_W, snd_W, snd_bias):
#        super(Ex5_model_emb, self).__init__()
#        self.fst_layer = torch.nn.Embedding(input, hidden, device=device, _weight=fst_W.T, padding_idx=pad)        
#        self.snd_linear_layer = torch.nn.Linear(hidden, 2, device=device)
#        self.snd_linear_layer.load_state_dict(OrderedDict([('weight',  snd_W), ('bias', snd_bias)]))
#        self.relu = torch.nn.ReLU(),
#    def forward(self, x):
#        x = self.fst_layer(x)
#        x = torch.sum(x, dim=1) 
#        x = torch.relu(x)
#        x = self.snd_linear_layer(x)
#        return x

# Summary models

In [ ]:
# Model BoW Freq
ex5_model = Ex5_model('bow_hist',len(vocab_train)+1, 128)
ex5_model.to(device)
#print(ex5_model.fst_layer.weight)
#print(ex5_model.fst_layer.weight.shape)

# Summary Model BoW Freq
print(ex5_model)

# Model Embedding
ex5_model_emb = Ex5_model('emb',len(vocab_train)+2, 128)
ex5_model_emb.to(device)
#print(ex5_model_emb.fst_layer.weight)
#print(ex5_model_emb.fst_layer.weight.shape)

# Summary Model Embedding
print(ex5_model_emb)


Ex5_model(
  (fst_layer): Linear(in_features=10001, out_features=128, bias=False)
  (snd_linear_layer): Linear(in_features=128, out_features=2, bias=False)
  (relu): ReLU()
)
Ex5_model(
  (fst_layer): Embedding(10002, 128, padding_idx=10001)
  (snd_linear_layer): Linear(in_features=128, out_features=2, bias=False)
  (relu): ReLU()
)


# Train loop

In [ ]:
n_epochs = 5
learningRate = 0.0001

# CrossEntropyLoss as loss function
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
def train_loop(dataloader_train, dataloader_val, hyperparameters, model):
    train_loss_a=[] 
    val_loss_a=[] 
    total_acc_a=[]
    # Gradient descent
    optimizer = torch.optim.Adam(model.parameters(), lr=hyperparameters['learning_rate'])
    min_val_loss = 10e9
    best_epoch = 0

    for epoch in range(hyperparameters['n_epochs']):
      # print(f'Model: {hyperparameters["mode"]} - Weights 1st layer')
      # print(model.fst_layer.weight.shape)
      # print(model.fst_layer.weight)
      train_loss = 0
      model.train()
      for x_train, y_train in dataloader_train:
            # transform to one dimention
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        #print(x_train)
        #print(x_train.shape)    
        #print(y_train) 
        #print(y_train.shape) 
            # predict 
        outputs = model(x_train)

            # batch loss
        batch_loss = criterion(outputs, y_train)

            # reset gradients, backpropagation, optimizer step and sum loss
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        train_loss += batch_loss.item()
            #print(f'{hyperparameters["name"]}_train/batch_loss: {batch_loss}')
        run[f'{hyperparameters["mode"]}_train/batch_loss'].log(batch_loss)

      train_loss = train_loss / len(dataloader_train.dataset)
      train_loss_a.append(train_loss)
        #print(f'Epoch {epoch} / {hyperparameters["name"]} train loss: {train_loss}')
      run[f'{hyperparameters["mode"]}_train/train_loss'].log(train_loss) 

        # Validation (end of epoch).
      total_loss = 0
      total_acc = 0
      model.eval()
      with torch.no_grad():
        for x_val, y_val in dataloader_val:
          x_val = x_val.to(device)
          y_val = y_val.to(device)

                # predict
          outputs = model(x_val)

                # batch loss
          batch_loss = criterion(outputs, y_val)
          preds = outputs.argmax(dim=1)

                # val acc
          batch_acc = (preds == y_val).sum()
          total_loss += batch_loss
          total_acc += batch_acc

      val_loss = total_loss / len(dataloader_val.dataset)
      val_loss_a.append(val_loss)
      run[f'{hyperparameters["mode"]}_val/val_loss'].log(val_loss)
      run[f'{hyperparameters["mode"]}_val/val_acuracy'].log(total_acc / len(dataloader_val.dataset))
      total_acc_a.append(total_acc / len(dataloader_val.dataset))
     
      print(f'Model: {hyperparameters["mode"]}, Epoch: {epoch+1}/{hyperparameters["n_epochs"]} - train_loss: {train_loss} - val_loss: {val_loss} - acc: {total_acc / len(dataloader_val.dataset)*100} %')

        # Save best model
      if val_loss < min_val_loss:
        torch.save(model.state_dict(), 'best_model.pt')
        min_val_loss = val_loss
        best_epoch = epoch
        print(f'Model: {hyperparameters["mode"]} - best model in epoch: {best_epoch+1}')
    return train_loss_a, val_loss_a, total_acc_a

In [ ]:
# Function to test acc
def predict(model, inputs):
    outputs = model(inputs)
    _, predicts = torch.max(outputs, 1)
    return predicts



# List to Dict

In [ ]:
# Transform list to dict
x_train = {num: i for num, i in enumerate(x_train)}
y_train = {num: i for num, i in enumerate(y_train)}
x_valid = {num: i for num, i in enumerate(x_valid)}
y_valid = {num: i for num, i in enumerate(y_valid)}
x_test = {num: i for num, i in enumerate(x_test)}
y_test = {num: i for num, i in enumerate(y_test)}

# BoW Hist sem Bias

In [ ]:
set_seeds()

In [ ]:
# Defining model
model_bow = Ex5_model('bow_hist', len(vocab_train)+1, 128)

# Saving weights list to use in embedding model
bow_weights = [model_bow.fst_layer.weight.clone(), model_bow.snd_linear_layer.weight.clone()]

model_bow.to(device)
print(f'bow_hist model - Weights shape 1st Layer: {model_bow.fst_layer.weight.shape}')
print(f'bow_hist model - Weights shape 2nd Layer: {model_bow.snd_linear_layer.weight.shape}')
#print(model_bow.fst_layer.weight)
#print(model_bow.snd_linear_layer.weight)

bow_hist model - Weights shape 1st Layer: torch.Size([128, 10001])
bow_hist model - Weights shape 2nd Layer: torch.Size([2, 128])


In [ ]:

hyperparameters = { "mode": "bow_hist",
          "learning_rate": 1e-4,
          "n_epochs": 5,
          "batch_size": 50,
          "hidden_size": 128 }

#model_bow = Ex5_model(hyperparameters['mode'], len(vocab_train)+1,  hyperparameters['hidden_size'])


train_ds = Ex5_ds(x_train, y_train, hyperparameters['mode'])
val_ds = Ex5_ds(x_valid, y_valid, hyperparameters['mode'])
dataloader_train = DataLoader(train_ds, batch_size=hyperparameters['batch_size'], shuffle=False)
dataloader_val = DataLoader(val_ds, batch_size=hyperparameters['batch_size'], shuffle=False)  


In [ ]:
train_loss_bow, val_loss_bow, acc_bow = train_loop(dataloader_train, dataloader_val, hyperparameters, model_bow)   

Model: bow_hist, Epoch: 1/5 - train_loss: 0.009897605769336223 - val_loss: 0.007173544727265835 - acc: 86.29999542236328 %
Model: bow_hist - best model in epoch: 1
Model: bow_hist, Epoch: 2/5 - train_loss: 0.005991089477390051 - val_loss: 0.006133963819593191 - acc: 87.63999938964844 %
Model: bow_hist - best model in epoch: 2
Model: bow_hist, Epoch: 3/5 - train_loss: 0.004725101256370544 - val_loss: 0.005948099307715893 - acc: 87.87999725341797 %
Model: bow_hist - best model in epoch: 3
Model: bow_hist, Epoch: 4/5 - train_loss: 0.003953803751245141 - val_loss: 0.006029663607478142 - acc: 87.95999908447266 %
Model: bow_hist, Epoch: 5/5 - train_loss: 0.003383462906628847 - val_loss: 0.006235369481146336 - acc: 87.75999450683594 %


In [ ]:
del train_ds
del val_ds
del dataloader_train
del dataloader_val

In [ ]:
test_ds = Ex5_ds(x_test, y_test, hyperparameters['mode'])
dataloader_test = DataLoader(test_ds, batch_size=hyperparameters['batch_size'], shuffle=False)  
total_acc = 0     
with torch.no_grad():
  for x_t, y_t in dataloader_test:
    x_t = x_t.to(device)
    #print(x_t.shape)
    y_t = y_t.to(device)
    #print(y_t.shape)
    outputs = model_bow(x_t)
    #print(outputs.shape)

    #preds = outputs > 0.5

    preds = outputs.argmax(dim=1)
    #print(preds.shape)

    # test acc
    batch_acc = (preds == y_t).sum()
    total_acc += batch_acc
  test_acc = total_acc / len(dataloader_test.dataset)

  print(f"BoW Hist Acc: {test_acc*100} %")    

BoW Hist Acc: 85.31999206542969 %


In [ ]:
del test_ds
del dataloader_test


# Embedding

In [ ]:
set_seeds()

In [ ]:
zeros = torch.zeros(128, 1) 
zeros = zeros.to(device) # need to compute in the same device

pad_weight = torch.cat([bow_weights[0], zeros], dim=1) # concatenating zeros to end of weights - PAD
emb_weights = OrderedDict([
    ('fst_layer.weight',  pad_weight.T),
    ('snd_linear_layer.weight',  bow_weights[1])])

In [ ]:
# Defining model
ex5_model_emb = Ex5_model('emb',len(vocab_train)+2, 128)

# Loading weights
ex5_model_emb.load_state_dict(emb_weights)

print(f'emb model - Weights shape 1st Layer: {ex5_model_emb.fst_layer.weight.shape}')
print(f'emb model - Weights shape 2nd Layer: {ex5_model_emb.snd_linear_layer.weight.shape}')

emb model - Weights shape 1st Layer: torch.Size([10002, 128])
emb model - Weights shape 2nd Layer: torch.Size([2, 128])


In [ ]:

hyperparameters = { "mode": "emb",
          "learning_rate": 1e-4,
          "n_epochs": 5,
          "batch_size": 50,
          "hidden_size": 128 }

#ex5_model_emb = Ex5_model(hyperparameters['mode'], len(vocab_train)+2, hyperparameters['hidden_size'])
ex5_model_emb.to(device)



train_ds = Ex5_ds(x_train, y_train, hyperparameters['mode'])
val_ds = Ex5_ds(x_valid, y_valid, hyperparameters['mode'])
dataloader_train = DataLoader(train_ds, batch_size=hyperparameters['batch_size'], shuffle=False)
dataloader_val = DataLoader(val_ds, batch_size=hyperparameters['batch_size'], shuffle=False)       
train_loss_emb, val_loss_emb, acc_emb = train_loop(dataloader_train, dataloader_val, hyperparameters, ex5_model_emb)   

Model: emb, Epoch: 1/5 - train_loss: 0.009897605761885643 - val_loss: 0.007173544727265835 - acc: 86.29999542236328 %
Model: emb - best model in epoch: 1
Model: emb, Epoch: 2/5 - train_loss: 0.005991089464724064 - val_loss: 0.006133963819593191 - acc: 87.63999938964844 %
Model: emb - best model in epoch: 2
Model: emb, Epoch: 3/5 - train_loss: 0.004725101243704557 - val_loss: 0.005948099307715893 - acc: 87.87999725341797 %
Model: emb - best model in epoch: 3
Model: emb, Epoch: 4/5 - train_loss: 0.003953803734853864 - val_loss: 0.006029663607478142 - acc: 87.95999908447266 %
Model: emb, Epoch: 5/5 - train_loss: 0.0033834629133343698 - val_loss: 0.006235369946807623 - acc: 87.75999450683594 %


In [ ]:
del train_ds
del val_ds
del dataloader_train
del dataloader_val

In [ ]:
test_ds = Ex5_ds(x_test, y_test, hyperparameters['mode'])
dataloader_test = DataLoader(test_ds, batch_size=hyperparameters['batch_size'], shuffle=False)  
total_acc = 0     
with torch.no_grad():
  for x_t, y_t in dataloader_test:
    x_t = x_t.to(device)
    #print(x_t.shape)
    y_t = y_t.to(device)
    #print(y_t.shape)
    outputs = ex5_model_emb(x_t)
    #print(outputs.shape)

    #preds = outputs > 0.5

    preds = outputs.argmax(dim=1)
    #print(preds.shape)

    # test acc
    batch_acc = (preds == y_t).sum()
    total_acc += batch_acc
  test_acc = total_acc / len(dataloader_test.dataset)

  print(f"Embedding Acc: {test_acc*100} %")   

Embedding Acc: 85.31999206542969 %


In [ ]:
del test_ds
del dataloader_test

#  Verify if BoW Freq. and Embedding models had same results

In [ ]:
print(f"Train loss BoW Freq. {train_loss_bow}")
print(f"Train loss Embedding {train_loss_emb}")

print(f"Val loss BoW Freq. {val_loss_bow}")
print(f"Val loss Embedding {val_loss_emb}")

print(f"Acc BoW Freq. {acc_bow}")
print(f"Acc Embedding {acc_emb}")


Train loss BoW Freq. [0.009897605769336223, 0.005991089477390051, 0.004725101256370544, 0.003953803751245141, 0.003383462906628847]
Train loss Embedding [0.009897605761885643, 0.005991089464724064, 0.004725101243704557, 0.003953803734853864, 0.0033834629133343698]
Val loss BoW Freq. [tensor(0.0072, device='cuda:0'), tensor(0.0061, device='cuda:0'), tensor(0.0059, device='cuda:0'), tensor(0.0060, device='cuda:0'), tensor(0.0062, device='cuda:0')]
Val loss Embedding [tensor(0.0072, device='cuda:0'), tensor(0.0061, device='cuda:0'), tensor(0.0059, device='cuda:0'), tensor(0.0060, device='cuda:0'), tensor(0.0062, device='cuda:0')]
Acc BoW Freq. [tensor(0.8630, device='cuda:0'), tensor(0.8764, device='cuda:0'), tensor(0.8788, device='cuda:0'), tensor(0.8796, device='cuda:0'), tensor(0.8776, device='cuda:0')]
Acc Embedding [tensor(0.8630, device='cuda:0'), tensor(0.8764, device='cuda:0'), tensor(0.8788, device='cuda:0'), tensor(0.8796, device='cuda:0'), tensor(0.8776, device='cuda:0')]


In [ ]:
assert torch.allclose(torch.tensor(train_loss_bow), torch.tensor(train_loss_emb), rtol=1e-05, atol=1e-08) and torch.allclose(torch.tensor(val_loss_bow), torch.tensor(val_loss_emb), rtol=1e-05, atol=1e-08) and torch.allclose(torch.tensor(acc_bow), torch.tensor(acc_emb), rtol=1e-05, atol=1e-08) 

In [ ]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!


Waiting for the remaining 15 operations to synchronize with Neptune. Do not kill this process.


All 15 operations synced, thanks for waiting!
